# Sea Surface Temperature

In [ ]:
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpd
from regions import boolean_mask, global_ocean, gl_ocean_rect, SST_index_bounds
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM
from xr_DataArrays import xr_AREA
from xr_regression import xr_linear_trends_2D, xr_linear_trend, ocn_field_regression

# global yearly avg. SST map and trends

In [ ]:
# %%time
# # ca. 4:30 min for ctrl/rcp, 1:25 for lpi
# # stacking files into one xr Dataset object
# for run in ['ctrl', 'rcp']:  # ['lpi', 'lpd']:
#     for i, (y,m,s) in enumerate(IterateOutputCESM('ocn', run, 'yrly', name='TEMP_PD')):
#         print(y)
#         da = xr.open_dataset(s, decode_times=False).TEMP[0,:,:]
#         da = da.drop(['z_t', 'ULONG', 'ULAT'])
#         da['TLAT' ] = da['TLAT' ].round(decimals=2)
#         da['TLONG'] = da['TLONG'].round(decimals=2)
#         del da.encoding["contiguous"]
#         ds = t2ds(da=da, name='SST', t=int(round(da.time.item())))
#         ds.to_netcdf(path=f'{path_samoc}/SST/SST_yrly_{run}_{y}.nc', mode='w')
     
#     combined = xr.open_mfdataset(f'{path_samoc}/SST/SST_yrly_{run}_*.nc',
#                                  concat_dim='time',
#                                  autoclose=True,
#                                  coords='minimal')
#     combined.to_netcdf(f'{path_samoc}/SST/SST_yrly_{run}.nc')
#     # remove extra netCDF files

In [ ]:
SST_yrly_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_ctrl.nc', decode_times=False)
SST_yrly_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_rcp.nc' , decode_times=False)
SST_yrly_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpd.nc' , decode_times=False)
SST_yrly_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_lpi.nc' , decode_times=False)

### mean CTRL SST

In [ ]:
xa    = SST_yrly_ctrl.mean(dim='time').where(MASK>0)
fn    = f'{path_results}/SST/SST_ctrl_mean'
cm    = discrete_cmap(17, cmocean.cm.thermal)
f, ax = map_robinson(xa=xa, domain='ocn_U', cmap=cm, minv=-2, maxv=32, label='SST [$^\circ$C]', filename=fn)

In [ ]:
%%time
# 2 min
SST_trend_ctrl, SST_trend_interc_ctrl = ocn_field_regression(SST_yrly_ctrl, run='ctrl')
SST_trend_rcp , SST_trend_interc_rcp  = ocn_field_regression(SST_yrly_rcp , run='rcp' )
SST_trend_lpd , SST_trend_interc_lpd  = ocn_field_regression(SST_yrly_lpd , run='lpd' )
SST_trend_lpi , SST_trend_interc_lpi  = ocn_field_regression(SST_yrly_lpi , run='lpi' )

In [ ]:
SST_yrly_lpd[:,100,100].plot()
(SST_trend_lpd[100,100]*SST_yrly_lpd.time + SST_trend_interc_lpd[100,100]).plot()

In [ ]:
cmap  = discrete_cmap(12, shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk'))

xa    = SST_trend_ctrl*100
fn    = f'{path_results}/SST/SST_trend_ctrl'
label = '100-299 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

xa    = SST_trend_rcp*100
fn    = f'{path_results}/SST/SST_trend_rcp'
label = '2000-2099 SST trend [K/century]'
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-2, maxv=4, label=label, filename=fn)

### make detrended yearly files

In [ ]:
# %%time
# # 3 min
# (SST_yrly_ctrl-(SST_trend_ctrl*SST_yrly_ctrl.time+SST_trend_interc_ctrl)).to_netcdf(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc')
# (SST_yrly_rcp -(SST_trend_rcp *SST_yrly_rcp .time+SST_trend_interc_rcp )).to_netcdf(f'{path_samoc}/SST/SST_yrly_detr_rcp.nc' )
# (SST_yrly_lpd -(SST_trend_lpd *SST_yrly_lpd .time+SST_trend_interc_lpd )).to_netcdf(f'{path_samoc}/SST/SST_yrly_detr_lpd.nc' )
# (SST_yrly_lpi -(SST_trend_lpi *SST_yrly_lpi .time+SST_trend_interc_lpi )).to_netcdf(f'{path_samoc}/SST/SST_yrly_detr_lpi.nc' )

In [ ]:
SST_yrly_detr_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_ctrl.nc', decode_times=False)
SST_yrly_detr_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_rcp.nc' , decode_times=False)
SST_yrly_detr_lpd  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_lpd.nc' , decode_times=False)
SST_yrly_detr_lpi  = xr.open_dataarray(f'{path_samoc}/SST/SST_yrly_detr_lpi.nc' , decode_times=False)

### hiatus trend

In [ ]:
SST_yrly_rcp[20:29,:,:]

In [ ]:
SST_trend_rcp_hiatus  = ocn_field_regression(SST_yrly_rcp[20:29,:,:])

In [ ]:
fn = f'{path_results}/SST/SST_trend_rcp_hiatus'
cmap = shifted_color_map(cmocean.cm.balance, start=.25, midpoint=0.5, stop=1., name='shrunk')
label = 'SST trend [K/century]'
xa = SST_trend_rcp_hiatus*100
f, ax = map_robinson(xa=xa, domain='ocn_T', cmap=cmap, minv=-4, maxv=8, label=label, filename=fn)

# Ocean basins and SST index regions

In [ ]:
for file in [file_ex_ocn_ctrl, file_ex_ocn_lpd]:
    REGION_MASK = xr.open_dataset(file, decode_times=False).REGION_MASK
    fig = plt.figure(figsize=(8,5))
    cmap = discrete_cmap(12, 'hsv')
    ax  = fig.add_subplot(1, 1, 1, projection=ccrs.EqualEarth(central_longitude=200))
    ax.set_position([.02,.05,.96,.93])
    cax, kw = mpl.colorbar.make_axes(ax, location='bottom', pad=0.03, shrink=0.8)

    im = ax.pcolormesh(REGION_MASK.TLONG,
                       REGION_MASK.TLAT,
                       REGION_MASK.where(REGION_MASK>0),
                       cmap=cmap,
                       vmin=.5, vmax=12.5,
                       transform=ccrs.PlateCarree() )

    gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=False)
    gl.ylocator = mpl.ticker.FixedLocator([-90, -60, -30, 0, 30, 60, 90])
    gl.xlocator = mpl.ticker.FixedLocator([-180, -120, -60, 0, 60, 120, 180])
    
    for mask in ['TPI1', 'TPI2', 'TPI3', 'SOM', 'AMO']:
        ax.add_patch(mpatches.Polygon(xy=rect_polygon(SST_index_bounds(mask)),
                                      facecolor='none',
                                      edgecolor='k',
                                      linewidth=2,
                                      zorder=2,
                                      transform=ccrs.PlateCarree()))

    ax.add_feature(cartopy.feature.LAND, zorder=3, edgecolor='black', facecolor='w')

    cbar = fig.colorbar(im, cax=cax, extend='min', **kw)
    cbar.set_ticks(np.arange(1,13))
    cbar.ax.tick_params(labelsize=14)
    label = cbar.set_label('region number', size=16)